In [1]:
import pandas as pd
import numpy as np

def pre_process(df: pd.DataFrame) -> pd.DataFrame:
    '''
    데이터 전처리 함수
    - 결측치 처리, 데이터 타입 변환, 매핑, 파생변수 생성, 기본 지표 계산까지 수행
    Args:
        df (pd.DataFrame): 원본 데이터프레임
    Returns:
        pd.DataFrame: 전처리 완료된 데이터프레임
    '''

    # 1. 결측치 처리
    df['사업장_시도'] = df['사업장_시도'].fillna('미정')
    df['사업장_시군구'] = df['사업장_시군구'].fillna('미정')

    # 2. 기준년월 변환 (YYYYMM → PeriodIndex[월])
    df['기준년월'] = pd.to_datetime(df['기준년월'].astype(str), format='%Y%m', errors='coerce')
    df['기준년월'] = df['기준년월'].dt.to_period('M')  # "2024-12" 형태 유지

    # 3. 대구/경북 여부 플래그
    df['대구경북권_여부'] = df['사업장_시도'].isin(['대구광역시', '경상북도'])

    # 4. 전담고객여부 매핑 (Y/N → True/False)
    df['전담고객여부'] = df['전담고객여부'].map({'N': False, 'Y': True})

    # 5. 좌수 범주 → 숫자 매핑
    seat_map = {
        "0개": 0, "1개": 1, "2개": 2,
        "2개초과 5개이하": 4, "5개초과 10개이하": 8,
        "10개초과 20개이하": 15.5, "20개초과 30개이하": 25.5,
        "30개초과 40개이하": 35.5, "40개초과 50개이하": 45.5,
        "50개 초과": 55
    }
    cols_to_map = [
        "요구불예금좌수","거치식예금좌수","적립식예금좌수",
        "수익증권좌수","신탁좌수","퇴직연금좌수",
        "여신_운전자금대출좌수","여신_시설자금대출좌수","신용카드개수"
    ]
    df[cols_to_map] = df[cols_to_map].replace(seat_map)

    # 6. 거래건수 범주 → 숫자 매핑
    txn_map = {
        "0건": 0, "1건": 1, "2건": 2,
        "2건초과 5건이하": 3, "5건초과 10건이하": 7,
        "10건초과 20건이하": 15, "20건초과 30건이하": 25,
        "30건초과 40건이하": 35, "40건초과 50건이하": 45,
        "50건 초과": 60
    }
    cols_to_map = [
        "외환_수출실적거래건수","외환_수입실적거래건수",
        "창구거래건수","인터넷뱅킹거래건수","스마트뱅킹거래건수",
        "폰뱅킹거래건수","ATM거래건수","자동이체거래건수"
    ]
    df[cols_to_map] = df[cols_to_map].replace(txn_map)

    # 7. 파생 변수 생성
    df['총거래건수'] = df[['창구거래건수','인터넷뱅킹거래건수','스마트뱅킹거래건수',
                         '폰뱅킹거래건수','ATM거래건수','자동이체거래건수']].sum(axis=1)
    df['총잔액'] = df[['요구불예금잔액','거치식예금잔액','적립식예금잔액']].sum(axis=1)
    df['여신잔액'] = df[['여신_운전자금대출잔액','여신_시설자금대출잔액']].sum(axis=1)
    df['외환거래총금액'] = df[['외환_수출실적금액','외환_수입실적금액']].sum(axis=1)
    df['수탁상품총잔액'] = df[['수익증권잔액','신탁잔액','퇴직연금잔액']].sum(axis=1)
    df['카드사용총금액'] = df[['신용카드사용금액','체크카드사용금액']].sum(axis=1)

    # 8. 여신이용률
    df['여신이용률'] = np.where(df['여신한도금액'] == 0, 0, df['여신잔액'] / df['여신한도금액'])

    # 9. 디지털 거래건수 & 비중
    df['디지털거래건수'] = df[['인터넷뱅킹거래건수','스마트뱅킹거래건수',
                            '폰뱅킹거래건수','자동이체거래건수']].sum(axis=1)

    def calc_digital_ratio(row: pd.Series) -> float:
        if row['총거래건수'] == 0:
            return 0.0 if row['디지털거래건수'] == 0 else pd.NA
        return row['디지털거래건수'] / row['총거래건수']

    df['디지털비중'] = df.apply(calc_digital_ratio, axis=1)

    # 10. 창구비중, 요구불회전율
    df['창구비중'] = np.where(df['총거래건수'] == 0, 0, df['창구거래건수'] / df['총거래건수'])
    df['요구불회전율'] = np.where(df['요구불예금잔액'] == 0, 0,
                               (df['요구불입금금액'] + df['요구불출금금액']) / df['요구불예금잔액'])

    # 11. 수치형 컬럼 강제 변환
    cols_numeric = ['여신이용률','디지털비중','창구비중','요구불회전율']
    for col in cols_numeric:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 12. 예대마진 계산
    loan_rate = 0.05
    deposit_rate = 0.02
    df["예대마진"] = (df["여신_운전자금대출잔액"] + df["여신_시설자금대출잔액"]) * loan_rate \
                   - df["총잔액"] * deposit_rate

    return df